## Парсинг HTML (hh.ru, superjob.ru)

#### Библиотеки для парсинга HTML и данных

In [538]:
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup as bs
from pprint import pprint

## Скачивание данных

#### 1. Подготовка параметров для скачивания

In [4]:
vacancy_name = input('Введите название вакансии для поиска: ')

Введите название вакансии для поиска: Директор по маркетингу


In [463]:
pages = int(input('Введите количество страниц для поиска: '))

Введите количество страниц для поиска: 1


In [464]:
vacancy_name, pages

('Директор по маркетингу', 1)

In [559]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
           'Accept': '*/*'}

In [611]:
hh_link = 'https://hh.ru'
sj_link = 'https://superjob.ru'

vacancies = []

for page in range(pages):
    #Поиск по странице hh.ru
    hh_params = {'page':str(page),
             'text':vacancy_name,
             'clusters':'true',
             'search_field':'name',
             'enable_snippets':'true'}
    response = requests.get(hh_link+'/search/vacancy', headers=headers, params=hh_params)
    soup = bs(response.text, 'html.parser')
    
    if response.ok:
        vacancies_block = soup.find('div',{'class':'vacancy-serp'})
        vacancies_list = vacancies_block.find_all('div',{'class':'vacancy-serp-item'})
        
        print(len(vacancies_list))
        
        for vacancy in vacancies_list:
            vacancy_data = {}
            vacancy_data['name'] = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'}).getText()
            vacancy_data['link'] = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'})['href']
            a = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
            if a == None:
                a = None
            else:
                a = a.getText()
                a = re.split('[\xa0 -]',a)
                vacancy_data['salary'] = a
                
                if len(a) == 4 and a[0] == 'от':
                    vacancy_data['min_salary'] = a[1]+a[2]
                if len(a) == 4 and a[0] == 'до':
                    vacancy_data['max_salary'] = a[1]+a[2]
                if len(a) > 4:
                    vacancy_data['min_salary'] = a[0]+a[1]
                    vacancy_data['max_salary'] = a[2]+a[3]
                    
                vacancy_data['currency'] = a[-1]
                    
            b = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-address'}).getText()
            vacancy_data['city'] = b.split(',')[0]
            vacancy_data['main_link'] = hh_link
            
            vacancies.append(vacancy_data)
            
    #Поиск по странице superjob.ru
    sj_params = {'page':str(page),
                 'keywords': vacancy_name}
    
    response = requests.get(sj_link+'/vacancy/search/', headers=headers, params=sj_params)
    soup = bs(response.text, 'html.parser')
    
    if response.ok:
        vacancies_block = soup.find('div',{'class':'_1ID8B'})
        vacancies_list = vacancies_block.find_all('div',{'class':'LvoDO'})
        
        print(len(vacancies_list))
        
        for vacancy in vacancies_list:
            vacancy_data = {}
            vacancy_data['name'] = vacancy.find('a',{'class':'_1UJAN'}).getText()
            vacancy_data['link'] = sj_link+vacancy.find('a',{'class':'_1UJAN'})['href']
            
            a = vacancy.find('span',{'class':'_2VHxz'}).getText()
            if a =='По договорённости':
                a = None
            else:
                a = re.split('[\xa0 -]',a)
                vacancy_data['salary'] = a
                
                if len(a) == 4 and a[0] == 'от':
                    vacancy_data['min_salary'] = a[1]+a[2]
                if len(a) == 4 and a[0] == 'до':
                    vacancy_data['max_salary'] = a[1]+a[2]
                if len(a) > 4:
                    vacancy_data['min_salary'] = a[0]+a[1]
                    vacancy_data['max_salary'] = a[2]+a[3]
                    
                vacancy_data['currency'] = a[-1]
                    
            b = vacancy.find('span',{'class':'clLH5'}).find_next_sibling().getText()
            vacancy_data['city'] = b.split(',')[0]
            vacancy_data['main_link'] = sj_link
            
            vacancies.append(vacancy_data)

50
18


In [612]:
vacancies_data = pd.DataFrame(vacancies)
vacancies_data

,name,link,salary,min_salary,max_salary,currency,city,main_link
0,Директор по маркетингу,https://hh.ru/vacancy/37582494?query=%D0%94%D0...,"[100, 000, 200, 000, руб.]",100000,200000,руб.,Москва,https://hh.ru
1,Директор по маркетингу,https://hh.ru/vacancy/37916053?query=%D0%94%D0...,"[500, 000, 1, 000, 000, KZT]",500000,1000,KZT,Нур-Султан,https://hh.ru
2,Директор по развитию и маркетингу,https://hh.ru/vacancy/38016442?query=%D0%94%D0...,"[от, 120, 000, руб.]",120000,NaN,руб.,Барнаул,https://hh.ru
3,Директор по маркетингу и развитию обменного пу...,https://hh.ru/vacancy/37415755?query=%D0%94%D0...,"[250, 000, 500, 000, руб.]",250000,500000,руб.,Санкт-Петербург,https://hh.ru
4,Директор по маркетингу,https://hh.ru/vacancy/37668079?query=%D0%94%D0...,"[100, 000, 100, 000, руб.]",100000,100000,руб.,Санкт-Петербург,https://hh.ru
...,...,...,...,...,...,...,...,...
63,Руководитель отдела маркетинга,https://superjob.ru/vakansii/rukovoditel-otdel...,"[от, 100, 000, руб.]",100000,NaN,руб.,Москва,https://superjob.ru
64,Коммерческий директор,https://superjob.ru/vakansii/kommercheskij-dir...,NaN,NaN,NaN,NaN,Москва,https://superjob.ru
65,Руководитель отдела продаж интернет-рекламы,https://superjob.ru/vakansii/rukovoditel-otdel...,NaN,NaN,NaN,NaN,Москва,https://superjob.ru
66,Руководитель отдела рекламы и PR,https://superjob.ru/vakansii/rukovoditel-otdel...,NaN,NaN,NaN,NaN,Москва,https://superjob.ru


## Проверка ошибок после парсинга в min&max зарплате 

Достаточно легко провести проверку max_salary < min_salary и исправить значения ориентируясь на колонку salary, которую можно удалить после проверки. Таких строк должно быть мало и они больше относятся к выбросам. Сотрудников с высокими зарплатами обычно ищут - без использования онлайн площадок.

In [618]:
vacancies_data.loc[vacancies_data['max_salary'] < vacancies_data['min_salary'], ['salary', 'min_salary', 'max_salary', 'currency']]

,salary,min_salary,max_salary,currency
1,"[500, 000, 1, 000, 000, KZT]",500000,1000,KZT
5,"[700, 000, 3, 000, 000, руб.]",700000,3000,руб.
8,"[80, 000, 200, 000, руб.]",80000,200000,руб.
11,"[70, 000, 120, 000, руб.]",70000,120000,руб.
28,"[60, 000, 140, 000, руб.]",60000,140000,руб.
